Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 2: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [755]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
from random import random, choice, randint, uniform
from copy import copy, deepcopy

import numpy as np

from dataclasses import dataclass

## The *Nim* and *Nimply* classes

In [756]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [757]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    #return true while there is at least one stick
    def __bool__(self):
        return sum(self._rows) > 0

    #overwrite str for show nim
    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [758]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [759]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


## Optimal Strategy

In [760]:
def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = choice(spicy_moves)
    return ply


## My EA strategy

In [761]:
# Evolution parameter
GENERATIONS = 5
POPULATION_SIZE = 50
OFFSPRING_SIZE = 10
MUTATION_PROBABILITY = 0.2
MUTATION_TRESHOLD = 0.2
XOVER_TRESHOLD = 0.5
TOURNAMENT_SIZE = 2
NUM_GAMES = 10

In [762]:
@dataclass
class Individual:
    fitness: tuple
    genotype: list[float]

def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

#mutation individual (:Individual   help to autocoplite code) (-> Individual   means that i return an Individual)
def mutate(ind: Individual) -> Individual: 
    offspring = deepcopy(ind)
    while True:
        #seleziono i due genomi da modificare e ciclo sinche sono uguali
        while True:
            to_change_1 = randint(0, len(ind.genotype)-1)
            to_change_2 = randint(0, len(ind.genotype)-1)

            if(to_change_1 != to_change_2):
                break

        #controllo di non rimuovere la soglia da un valore che è gia a 0 o aggiungerla a un valore che è gia 1
        if offspring.genotype[to_change_1] - MUTATION_TRESHOLD >= 0 and offspring.genotype[to_change_2] + MUTATION_TRESHOLD <= 1:
            offspring.genotype[to_change_1] -= MUTATION_TRESHOLD
            offspring.genotype[to_change_2] += MUTATION_TRESHOLD
            break

    offspring.genotype = [round(element, 2) for element in offspring.genotype]
    return offspring

def adjust_to_one(vector):
    # Calcola la somma attuale del vettore
    current_sum = sum(vector)

    # Trova l'indice dell'elemento più grande
    max_index = vector.index(max(vector))

    # Calcola la differenza da aggiungere all'elemento più grande
    difference = 1 - current_sum

    # Aggiungi la differenza all'elemento più grande
    vector[max_index] += round(difference, 2)

    return vector

def one_cut_xover(ind1: Individual, ind2: Individual):

    new_vet = [(a + b) / 2 for a, b in zip(ind1.genotype, ind2.genotype)]
    #round at second value
    new_vet = [round(element, 2) for element in new_vet]

    offspring = Individual(fitness=None, genotype=new_vet)

    return offspring
    #return deepcopy(ind1)

In [763]:
def all_from_min(state: Nim) -> Nimply:
    #print("all_from_min")
    """Pick always the maximum possible number of the row with lowest stick"""
    actual_status = [(index, val) for index, val in enumerate(state._rows) if val > 0]
    tupla_min_sticks = min(actual_status, key=lambda x: x[1])    
    return Nimply(*tupla_min_sticks )

def all_from_max(state: Nim) -> Nimply:
    #print("all_from_max")
    """Pick always the maximum possible number of the row with highest stick"""
    actual_status = [(index, val) for index, val in enumerate(state._rows) if val > 0]
    tupla_min_sticks = max(actual_status, key=lambda x: x[1])    
    return Nimply(*tupla_min_sticks )

def one_from_min(state: Nim) -> Nimply:
    #print("one_from_min")
    """Pick always one from the row with the lowest number of sticks"""
    actual_status = [(index, val) for index, val in enumerate(state._rows) if val > 0]
    tupla_min_sticks = min(actual_status, key=lambda x: x[1])
    return Nimply(*(tupla_min_sticks[0], 1) )

def one_from_max(state: Nim) -> Nimply:
    #print("one_from_max")
    """Pick always one from the row with the highest number of sticks"""
    actual_status = [(index, val) for index, val in enumerate(state._rows) if val > 0]
    tupla_min_sticks = max(actual_status, key=lambda x: x[1])
    return Nimply(*(tupla_min_sticks[0], 1) )

def last_shot(state: Nim) -> Nimply:
    #print("last_shot")
    counter_1 = state._rows.count(1)

    elemento_da_trovare = next((x for x in state._rows if x != 0 and x != 1), 1)
    indice = state._rows.index(elemento_da_trovare)

    if((counter_1 % 2) == 1):
        return Nimply(*(indice, state._rows[indice]) )
    elif((counter_1 % 2) == 0) :        
        return Nimply(*(indice, state._rows[indice]-1) )

def two_from_max(state):
    #print("two_from_max")
    actual_status = [(index, val) for index, val in enumerate(state._rows) if val > 0]
    tupla_min_sticks = max(actual_status, key=lambda x: x[1])
    tot=2
    if(tupla_min_sticks[1] < tot):
        tot = 1
    return Nimply(*(tupla_min_sticks[0], tot) )

def almost_all_from_max(state):
    #print("almost_all_from_max")
    actual_status = [(index, val) for index, val in enumerate(state._rows) if val > 0]
    tupla_min_sticks = max(actual_status, key=lambda x: x[1])
    tot = tupla_min_sticks[1] - 1
    if(tot == 0):
        tot = 1
    return Nimply(*(tupla_min_sticks[0], tot) )


my_strategies = [all_from_min, all_from_max, one_from_min, one_from_max, last_shot, two_from_max, almost_all_from_max]

def adaptive_strategies(state: Nim, ind: Individual):
   
    # Genera un numero casuale tra 0 e 1
    random_number = uniform(0, 1)

    # Calcola le probabilità cumulative
    cumulative_probabilities = []
    sum_tmp = 0
    for el in ind.genotype:
        sum_tmp += el
        cumulative_probabilities.append(sum_tmp)
        
    # Trova l'indice della prima probabilità cumulativa maggiore del numero casuale generato
    selected_index = next(i for i, p in enumerate(cumulative_probabilities) if p >= random_number)

    # Chiama la funzione corrispondente all'indice selezionato
    return my_strategies[selected_index](state)

In [764]:
def fitness(state: Nim, phenotype: Individual):
    total_wins = 0

    #the strategies of players
    strategy = (adaptive_strategies, optimal)

    for _ in range(NUM_GAMES):
        #create new game
        nim = deepcopy(state) # Nim(5)  # Crea un nuovo gioco

        player = 0
        while nim:
            #get the move (of player x) to do with the associated strategy
            # pass the nim for make the choose
            if(strategy[player] == adaptive_strategies):
                ply = strategy[player](nim, phenotype)
                strategy = (optimal, optimal)
            else:
                ply = strategy[player](nim)
            # do the move that is previous calcolated
            nim.nimming(ply)
            #change the player turn
            player = 1 - player

        # Se il giocatore 0 (la strategia) ha vinto, incrementa il conteggio delle vittorie
        if player == 0:
            total_wins += 1

    # La fitness è il numero totale di partite vinte
    return total_wins

In [765]:
def genera_vettore(n):
    vet = []
    tot = 0
    for i in range(n):
        dado = round(uniform(0, 1), 1)
        if(tot+dado <= 1):
            vet.append(dado)
            tot = tot + dado
        else: 
            vet.append(1-tot)
            for c in range(i+1, n):
                vet.append(0)
            break
    return vet

def adaptive(state_original: Nim) -> Nimply:

    state = deepcopy(state_original)
    
    #initial population
    population = [ Individual(fitness=None, genotype=genera_vettore(len(my_strategies))) for _ in range(POPULATION_SIZE) ]
    #calculate fitness
    for i in population:
        i.fitness = fitness(state, i)

    for gen in range(GENERATIONS):
        # Genera una nuova popolazione combinando e mutando i migliori individui
        offspring = list()
        for _ in range(OFFSPRING_SIZE):

            # Applica la mutazione
            if random() < MUTATION_PROBABILITY:
                #mutation  # add more clever mutation
                parent = select_parent(population)
                child = mutate(parent)
            else:
                #xOver  #add more xOver
                parent1 = select_parent(population)
                parent2 = select_parent(population)
                child = one_cut_xover(parent1,parent2)            

            offspring.append(child)
        
        # Aggiorna la popolazione
        population = offspring

        #valutate individuals
        # Evaluate the fitness of each individual in the population
        for individual in population:
            individual.fitness = fitness(state, individual)

        population.extend(offspring)
        # Select the best individuals
        population.sort(key = lambda i: i.fitness, reverse=True)
        
        #Survival Selection
        population = population[:POPULATION_SIZE]

    # Ottieni il miglior individuo
    best_individual = population[0]
    
    #return adaptive_strategies(state, best_individual)
    index_of_best = best_individual.genotype.index(max(best_individual.genotype))
    '''
    print("start STUFF")
    print(state)
    print(best_individual.genotype)
    print(index_of_best)
    print(my_strategies[index_of_best])
    print("end STUFF")
    '''
    return my_strategies[index_of_best](state)

## Oversimplified match

In [766]:
logging.getLogger().setLevel(logging.INFO)

#the strategies of players
strategy = (adaptive, optimal)
win_0 = 0
win_1 = 0
for i in range(100):

    #create nim game
    nim = Nim(5)
    logging.info(f"init : {nim}")

    #choose the player who will start
    player = 0


    #play the game while there are stiks to pic
    while nim:
        #get the move (of player x) to do with the associated strategy
        # pass the nim for make the choose
        ply = strategy[player](nim)
        logging.info(f"ply: player {player} plays {ply}")
        # do the move that is previous calcolated
        nim.nimming(ply)
        logging.info(f"status: {nim}")
        #change the player turn
        player = 1 - player
    logging.info(f"status: Player {player} won!")
    if(player == 0):
        win_0 = win_0 + 1
    else:
        win_1 = win_1 + 1

    print("win_0: ",win_0,"win_1: ", win_1)


INFO:root:init : <1 3 5 7 9>


INFO:root:ply: player 0 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 3 5 7 9>
INFO:root:ply: player 1 plays Nimply(row=3, num_objects=3)
INFO:root:status: <0 3 5 4 9>


start STUFF
<1 3 5 7 9>
[0.41, 0.29, 0.05, 0.0, 0.05, 0.2, 0.0]
0
<function all_from_min at 0x000001A539F3FE20>
end STUFF


INFO:root:ply: player 0 plays Nimply(row=1, num_objects=3)
INFO:root:status: <0 0 5 4 9>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=2)
INFO:root:status: <0 0 3 4 9>


start STUFF
<0 3 5 4 9>
[0.42, 0.21, 0.06, 0.0, 0.05, 0.0, 0.25]
0
<function all_from_min at 0x000001A539F3FE20>
end STUFF


...

INFO:root:ply: player 0 plays Nimply(row=4, num_objects=9)
INFO:root:status: <0 3 5 1 0>
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=1)
INFO:root:status: <0 2 5 1 0>


start STUFF
<0 3 5 1 9>
[0.31, 0.37, 0.14, 0.02, 0.06, 0.05, 0.05]
1
<function all_from_max at 0x000001A539F3F520>
end STUFF


INFO:root:ply: player 0 plays Nimply(row=3, num_objects=1)
INFO:root:status: <0 2 5 0 0>
INFO:root:ply: player 1 plays Nimply(row=2, num_objects=1)
INFO:root:status: <0 2 4 0 0>


start STUFF
<0 2 5 1 0>
[0.34, 0.3, 0.08, 0.0, 0.08, 0.2, 0.0]
0
<function all_from_min at 0x000001A539F3FE20>
end STUFF


KeyboardInterrupt: 